In [0]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('ch5').getOrCreate()

In [0]:
from pyspark.sql.types import LongType
def cubed(a):
  return a**3
spark.udf.register("cubing",cubed,LongType())
spark.range(1,9).createOrReplaceTempView('udf_test')

In [0]:
spark.sql("""select id, cubing(id) from udf_test""").show()

+---+----------+
 id|cubing(id)|
+---+----------+
 1| 1|
 2| 8|
 3| 27|
 4| 64|
 5| 125|
 6| 216|
 7| 343|
 8| 512|
+---+----------+

In [0]:
from pyspark.sql.types import *
schema = StructType([StructField('celsius',ArrayType(IntegerType()))])
t_list=[[35, 36, 32, 30, 40, 42, 38]], [[31, 32, 34, 55, 56]]
t_c=spark.createDataFrame(t_list,schema)
t_c.createOrReplaceTempView('tC')
t_c.show()

+--------------------+
 celsius|
+--------------------+
[35, 36, 32, 30, ...|
[31, 32, 34, 55, 56]|
+--------------------+

In [0]:
#celsius to fahrenheit
spark.sql('''select celsius, transform(celsius,t->((t*9) div 5)+32) as fahrenheit from tC''').show()
#NOTE: im yet to see the alternate approach for above, i.e im not able to create the same using df approach.

+--------------------+--------------------+
 celsius| fahrenheit|
+--------------------+--------------------+
[35, 36, 32, 30, ...|[95, 96, 89, 86, ...|
[31, 32, 34, 55, 56]|[87, 89, 93, 131,...|
+--------------------+--------------------+

In [0]:
#NOTE: below code does not work
#t_c2= t_c.transform(t_c['celsius'],t=>((t*9)/5)+32)
#t_c2.toDF.show()

File "<command-1314059222953876>" , line 1 
 t_c2= t_c.transform(t_c['celsius'],t=>((t*9)/5)+32) 
 ^
 SyntaxError : invalid syntax

In [0]:
#Filter temperatures > 38C for array of temperatures
spark.sql("""select celsius, filter(celsius, t->t>38) as highTemp from tC""").show()

+--------------------+--------+
 celsius|highTemp|
+--------------------+--------+
[35, 36, 32, 30, ...|[40, 42]|
[31, 32, 34, 55, 56]|[55, 56]|
+--------------------+--------+

In [0]:
#Is there a temperature of 38C in the array of temperatures
spark.sql("""select celsius, exists(celsius,t->t=38) as has38 from tC""").show()

+--------------------+-----+
 celsius|has38|
+--------------------+-----+
[35, 36, 32, 30, ...| true|
[31, 32, 34, 55, 56]|false|
+--------------------+-----+